In [ ]:
import os
import h5py
import re
from ase.data import chemical_symbols

one_symbol = []
two_symbol = []
for symbol in chemical_symbols[1::]:
    if len(symbol) == 1:
        one_symbol.append(symbol)
    elif len(symbol) == 2:
        two_symbol.append(symbol)
# print(one_symbol)
# print(two_symbol)

In [ ]:
def parse_elements(element: str, verbose: bool = False) -> list[str]:
    parsed = []
    idx = 0
    while idx < len(element):
        if verbose:
            print(f"......{idx}")
        if f"{element[idx : idx + 2]}" in two_symbol:
            if verbose:
                print(f">>>>, 2 {element[idx : idx + 2]}")
            match = re.search(r"^[0-9]+", element[idx + 2 : :])
            if match:
                if verbose:
                    print(f"two_symbol {match}")
                for i in range(0, int(match.group(0))):
                    parsed.append(f"{element[idx : idx + 2]}")
                if verbose:
                    print(f"parsed, 2, ye match {parsed}, {match.group(0)}")
                idx += 1 + len(f"{match.group(0)}")
            else:
                parsed.append(f"{element[idx : idx + 2]}")
                if verbose:
                    print(f"parsed, 2, no match{parsed}")
                idx += 2
        elif f"{element[idx]}" in one_symbol:
            if verbose:
                print(f">>>>, 1 {element[idx]}")
            match = re.search(r"^[0-9]+", element[idx + 1 : :])
            if match:
                if verbose:
                    print(f"one_symbol {match}")
                for i in range(0, int(match.group(0))):
                    parsed.append(f"{element[idx]}")
                if verbose:
                    print(f"parsed {parsed}, {match.group(0)}")
                idx += 1 + len(f"{match.group(0)}")
            else:
                if idx + 2 < len(element):
                    if f"{element[idx : idx + 2]}" in two_symbol:
                        if verbose:
                            print(f">>>> {element[idx]}")
                        match = re.search(r"^[0-9]+", element[idx + 2 : :])
                        if match:
                            if verbose:
                                print(f"two_symbol {match}")
                            for i in range(0, int(match.group(0))):
                                parsed.append(f"{element[idx]}")
                            if verbose:
                                print(f"parsed {parsed}, {match.group(0)}")
                            idx += 2 + len(f"{match.group(0)}")
                        else:
                            parsed.append(f"{element[idx : idx + 2]}")
                            if verbose:
                                print(f"parsed {parsed}")
                            idx += 2
                    else:
                        parsed.append(f"{element[idx]}")
                        if verbose:
                            print(f"parsed {parsed}")
                        idx += 1
                else:
                    parsed.append(f"{element[idx]}")
                    if verbose:
                        print(f"parsed {parsed}")
                    idx += 1
        else:
            if verbose:
                print(f">>>>>>>>>>>{element[idx]}")
            idx += 1
    return parsed


# parse_elements("HO2", verbose=True)

In [ ]:
project_path = "CHANGEME"  # R40_110290Au.hdf5"
cases = set()
for root, dirs, files in os.walk(project_path):
    for filename in files:
        fpath = os.path.join(root, filename)
        # print(fpath)
        with h5py.File(fpath, "r") as h5r:
            for key in h5r:
                # print(key)
                if key == "ranges":
                    for grpnm in h5r["ranges"]:
                        # print(f"ion{int(grpnm.replace("range_", ""))}")
                        for attribute_name in h5r["ranges"][grpnm].attrs:
                            cases.add(f"""{h5r["ranges"][grpnm].attrs["element"]}""")

In [ ]:
for entry in cases:
    print(f"{entry} --> {parse_elements(entry)}")
    # print(f"{parse_elements(entry)}")